In [287]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt2
import numpy as np
import math
from random import uniform
from random import randint 
import random
import pandas as pd
from scipy.cluster.vq import kmeans,vq,whiten
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import time
from math import sqrt
from sklearn.model_selection import cross_val_score
# import matplotlib.pyplot as ax1
import matplotlib.pyplot as ax2
import matplotlib.pyplot as ax3
import matplotlib.pyplot as ax4
import matplotlib.pyplot as ax5

import matplotlib.pyplot as plt_3d
from mpl_toolkits.mplot3d.art3d import Poly3DCollection



In [288]:
#INITIALIZATION
population_size = 50
# population_size = 10
max_generation = 100
# max_generation = 250
# max_generation = 10
_lambda = 1.5
dimension = 3
max_domain = 1000
min_domain = 0
step_size_cons = 0.01
Pa = 0.25

In [289]:
#OBJECTIVE FUNCTION
def schwefel(path, X_train, y_train):
    regressor = SVR(kernel='rbf', gamma=path[-1], C=path[-3], epsilon=path[-2])
    regressor.fit(X_train,y_train)
    score = sqrt(mean_squared_error(y_train, regressor.predict(X_train)))
    return score


In [290]:
#LEVY FLIGHT
def levy_flight(Lambda):
    sigma1 = np.power((math.gamma(1 + Lambda) * np.sin((np.pi * Lambda) / 2)) \
                      / math.gamma((1 + Lambda) / 2) * np.power(2, (Lambda - 1) / 2), 1 / Lambda)
    sigma2 = 1
    u = np.random.normal(0, sigma1, size=dimension)
    v = np.random.normal(0, sigma2, size=dimension)
    step = u / np.power(np.fabs(v), 1 / Lambda)

    return step

In [291]:
class Indivisual:
    def __init__(self):
         # reading data from file
#         dataset = pd.read_csv('hidrat_data.csv')
#         X = dataset.iloc[:,[1,2,4,5]].values
#         Y = dataset.iloc[:,[3]].values
#         X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)
#         X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1234)
#         st_x = StandardScaler()
#         self.X_train=st_x.fit_transform(X_train)
#         st_y = StandardScaler()
#         self.y_train=st_y.fit_transform(y_train)
#         X_test=st_x.fit_transform(X_test)
#         y_test=st_y.fit_transform(y_test)

    # vector quantization
        dataset = pd.read_csv('hidrat_data.csv')
        dataset = dataset.iloc[:,[1,2,3,4,5]].values
 # whitening of data
#         dataset = whiten(dataset)
        scale = StandardScaler()
        dataset=scale.fit_transform(dataset)
# computing K-Means with K = 10 (10 clusters)
        centroids,_ = kmeans(dataset,10)
# assign each sample to a cluster
        clx,_ = vq(dataset,centroids)
# The coordinates of each data are replaced by the coordinates of the center of the cluster in train dataset
        dataset_vq = np.zeros((len(clx),centroids.shape[1]))
        for i in range(len(clx)):
            dataset_vq[i] = centroids[clx[i]]
        dataset_vq= scale.inverse_transform(dataset_vq)
        
        X = dataset_vq[:,[0,1,3,4]]
        Y = dataset_vq[:,[2]]

        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)
        self.X_train, X_val, self.y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1234)
        
        st_x = StandardScaler()
        self.X_train=st_x.fit_transform(self.X_train)
        st_y = StandardScaler()
        self.y_train=st_y.fit_transform(self.y_train)
        X_test=st_x.fit_transform(X_test)
        y_test=st_y.fit_transform(y_test)
        
        self.__position = np.random.rand(dimension) * (max_domain - min_domain) + min_domain
        self.__fitness = schwefel(self.__position,X_train, y_train)

    def get_position(self):
        return self.__position

    def get_fitness(self):
        return self.__fitness

    def set_position(self, position):
        self.__position = position

    def set_fitness(self, fitness):
        self.__fitness = fitness

    def abandon(self):
        # abandon some variables
        for i in range(len(self.__position)):
            p = np.random.rand()
            if p < Pa:
                self.__position[i] = np.random.rand() * (max_domain - min_domain) + min_domain
        self.__fitness = schwefel(self.__position,self.X_train,self.y_train)


In [292]:
# def main():
    
test_mse = []
gammas=[]
c_values = []
epsilons=[]
t_array = []
best_fit_array = []
train_acc = []
test_acc = []
# reading data from file
# dataset = pd.read_csv('hidrat_data.csv')
# X = dataset.iloc[:,[1,2,4,5]].values
# Y = dataset.iloc[:,[3]].values
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1234)
# st_x = StandardScaler()
# X_train=st_x.fit_transform(X_train)
# st_y = StandardScaler()
# y_train=st_y.fit_transform(y_train)
# X_test=st_x.fit_transform(X_test)
# y_test=st_y.fit_transform(y_test)

# vector quantization
dataset = pd.read_csv('hidrat_data.csv')

dataset = dataset.iloc[:,[1,2,3,4,5]].values
scale = StandardScaler()
dataset=scale.fit_transform(dataset)
 # whitening of data
# dataset = whiten(dataset)
# computing K-Means with K = 10 (10 clusters)
centroids,_ = kmeans(dataset,10)
# assign each sample to a cluster
clx,_ = vq(dataset,centroids)
# The coordinates of each data are replaced by the coordinates of the center of the cluster in train dataset
dataset_vq = np.zeros((len(clx),centroids.shape[1]))
for i in range(len(clx)):
    dataset_vq[i] = centroids[clx[i]]
dataset_vq= scale.inverse_transform(dataset_vq)

X = dataset_vq[:,[0,1,3,4]]
Y = dataset_vq[:,[2]]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1234)

st_x = StandardScaler()
X_train=st_x.fit_transform(X_train)
st_y = StandardScaler()
y_train=st_y.fit_transform(y_train)
X_test=st_x.fit_transform(X_test)
y_test=st_y.fit_transform(y_test)

In [293]:
    #CS
    #RANDOMLY CREATING HOSTS
cs_list = []
for i in range(population_size):
        cs_list.append(Indivisual())


    #SORT TO GET THE BEST FITNESS
cs_list = sorted(cs_list, key=lambda ID: ID.get_fitness())

best_fitness = cs_list[0].get_fitness()
best_position = cs_list[0].get_position()

#     fig = plt.figure()

    #INITIAL POPULATION DISTRIBUTION
#     ax1 = fig.add_subplot(131)

# fig = plt2.figure()
# ax1 = fig.add_subplot(111, projection='3d')
# for i in range(population_size):
#         ax1.scatter([cs_list[i].get_position()[0]], [cs_list[i].get_position()[1]],[cs_list[i].get_position()[2]])
# ax1.set_title('Initial Population Distributtion')
# ax1.set_xlabel('C')
# ax1.set_ylabel('epsilon')
# ax1.set_zlabel('gamma')
# fig.show()

C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d ar

In [294]:
#     ax3 = fig.add_subplot(133)
#     ax4 = fig.add_subplot(133)

t = 1
#     while(best_fitness > 0.009):
model_predict = []
while(t < max_generation):
    model_predict = []
        #GENERATING NEW SOLUTIONS
    for i in range(population_size):

            #CHOOSING A RANDOM CUCKOO (say i)
        i = np.random.randint(low=0, high=population_size)

            #SETTING ITS POSITION USING LEVY FLIGHT
        position = cs_list[i].get_position()+(step_size_cons*levy_flight(_lambda))

            # Simple Boundary Rule
        for i in range(len(position)):
            if position[i] > max_domain:
                position[i] = max_domain
            if position[i] < min_domain:
                position[i] = min_domain

        cs_list[i].set_position(position)
        cs_list[i].set_fitness(schwefel(cs_list[i].get_position(),X_train,y_train))

            #CHOOSING A RANDOM HOST (say j)
        j = np.random.randint(0, population_size)
        while j == i:  # random id[say j] ≠ i
            j = np.random.randint(0, population_size)

            #RELAXATION
        if cs_list[j].get_fitness() > cs_list[i].get_fitness():
            cs_list[j].set_position(cs_list[i].get_position())
            cs_list[j].set_fitness(cs_list[i].get_fitness())

        #SORT (to Keep Best)
    cs_list = sorted(cs_list, key=lambda ID: ID.get_fitness())

        #ABANDON SOLUTION (exclude the best)
    for a in range(1, population_size):
        r = np.random.rand()
        if (r < Pa):
            cs_list[a].abandon()

        #RANKING THE CS LIST
    cs_list = sorted(cs_list, key=lambda ID: ID.get_fitness())

        #FIND THE CURRENT BEST
    if cs_list[0].get_fitness() < best_fitness:
        best_fitness = cs_list[0].get_fitness()
        best_position=cs_list[0].get_position()
    
    
            
#       mse for test set
    c_value = 0
    if best_position[-3] <=0:
        c_value = 0.00000001
    else:
        c_value = best_position[-3]
    regressor = SVR(kernel='rbf', gamma=best_position[-1], C=c_value, epsilon=best_position[-2])
    regressor.fit(X_train,y_train)
    model_predict = regressor.predict(X_test)
    print(r2_score(y_train, regressor.predict(X_train)))
    print(r2_score(y_test, regressor.predict(X_test)))
    test_MSE = sqrt(mean_squared_error(y_test, regressor.predict(X_test)))
    train_r2= r2_score(y_train, regressor.predict(X_train))
    test_r2 = r2_score(y_test, regressor.predict(X_test))
    train_acc.append(train_r2)
    test_acc.append(test_r2)

    test_mse.append(test_MSE)
    gammas.append(best_position[-1])
    c_values.append(c_value)
    if best_position[-2]==0:
        epsilons.append(0.001)
    else:
        epsilons.append(best_position[-2])
        
        
        #PRINTING SOLUTION IN EACH ITERATION
    print("iteration =", t, " best_fitness =", best_fitness,"train_acc=",train_r2,"pos=", best_position,"test mse=",test_MSE,"test_acc =",test_r2)
        
    t_array.append(t)
    best_fit_array.append(best_fitness)
#       mse for test set plotting
#         ax4.scatter(t, test_MSE)
        
        #FITNESS PLOTTING
#         ax3.scatter(t, best_fitness)

    t += 1
            

C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d ar

-2.407277166412085
-2.407277166412086
iteration = 1  best_fitness = 1.8094185713924453 train_acc= -2.407277166412085 pos= [599.96667784 620.65390178 922.90341974] test mse= 1.8458811355046907 test_acc = -2.407277166412086


C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d ar

-2.407277166412085
-2.407277166412086
iteration = 2  best_fitness = 1.8094185713924453 train_acc= -2.407277166412085 pos= [599.96667784 620.65390178 922.90341974] test mse= 1.8458811355046907 test_acc = -2.407277166412086


C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d ar

...

-2.407277166412085
-2.407277166412086
iteration = 14  best_fitness = 1.8090843150106455 train_acc= -2.407277166412085 pos= [551.04385633 739.1053374  283.73535268] test mse= 1.8458811355046907 test_acc = -2.407277166412086


C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d ar

0.7070189250555551
0.04547157378396238
iteration = 15  best_fitness = 0.5411511166562581 train_acc= 0.7070189250555551 pos= [412.89262006   0.62195433 283.76854935] test mse= 0.9769997063541205 test_acc = 0.04547157378396238


C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Asus\Anaconda3\envs\DeepLearning_Env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d ar

ValueError: C <= 0

In [ ]:


    #GRAPH FOR FITNESS
ax3.plot(t_array,best_fit_array,label = 'trin RMSE')
ax3.plot(t_array,train_acc,label = 'train R^2')
ax3.title('train RMSE/R^2')
ax3.xlabel('iteration')
ax3.ylabel('RMSE/R^2')
ax3.legend()
ax3.show()
    #GRAPH FOR FITNESS
ax4.plot(t_array, test_mse, label = 'test RMSE')
ax4.plot(t_array, test_acc,label = 'test R^2')
ax4.title('test RMSE/R^2')
ax4.xlabel('iteration')
ax4.ylabel('RMSE/R^2')
ax4.legend()
ax4.show()



    # FINAL POPULATION DISTRIBUTION
#     ax2 = fig.add_subplot(132)
# fig2 = plt.figure()
# ax2 = fig2.add_subplot(111, projection='3d')
# for i in range(population_size):
#         ax2.scatter([cs_list[i].get_position()[0]], [cs_list[i].get_position()[1]],[cs_list[i].get_position()[2]])
# ax2.set_title('Final Population Distributtion after '+str(t)+' iterations')
# ax2.set_xlabel('C')
# ax2.set_ylabel('epsilon')
# ax2.set_zlabel('gamma')

    #SHOWING GRAPH
# fig2.show()
    

In [ ]:
test_num = []
for i in range(len(y_test)):
    test_num.append(i)
test_num = test_num[::20]
model_predict = st_y.inverse_transform(model_predict)
y_test2 = st_y.inverse_transform(y_test)
model_predict2= model_predict[::20]
y_test_each20 = y_test2[::20]
fig = plt.figure(figsize=(30, 10))
ax2 = fig.add_subplot(111)
ax2.plot(test_num,model_predict2, label='Predicted value')
ax2.scatter(test_num,y_test_each20 , label = 'Actual value',color= 'red')
ax2.set_title('Comparison of actual values with predected values for the CS-VQ-SVR model')
ax2.set_xlabel('Data number',size = 20)
ax2.set_ylabel('Pressure (Psi)',size = 20)
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)
ax2.legend()
fig.show()

In [ ]:

# print(len(gammas))
# print(len(c_values))
# print(len(epsilons))
from matplotlib import cm
x1 = np.asarray(gammas)
x2 = np.asarray(c_values)
# x3 =  np.asarray(epsilons)
X1, X2 = np.meshgrid(x1, x2)
X3 = []
X4 = []
for i in range(len(X1)):
    x3 = []
    x4 = []
    for j in range(len(X1)):
        if i ==j:
            x3.append(epsilons[i])
            x4.append(test_acc[i])
        else:
            x3.append(0)
            x4.append(0)
    X4.append(x4)
    X3.append(x3)
X3 = np.asarray(X3)
X4 = np.asarray(X4)
# ax6.set_zlim([-10,1000])
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
color_dimension = X4 # change to desired fourth dimension
minn, maxx = color_dimension.min(), color_dimension.max()
norm = matplotlib.colors.Normalize(minn, maxx)
m = plt_3d.cm.ScalarMappable(norm=norm, cmap='jet')
m.set_array([])
fcolors = m.to_rgba(color_dimension)
fig6 = plt_3d.figure(figsize=(8, 8))
ax6 = fig6.gca(projection='3d')
surf = ax6.plot_surface(X1, X2, X3,facecolors=fcolors,rstride=1, edgecolors='k', lw=0.6, cmap='plasma',cstride=1,vmin=minn, vmax=maxx)
fig.colorbar(surf, ax = ax6,
             shrink = 0.5)
ax6.set_xlabel('gamma')
ax6.set_ylabel('c')
ax6.set_zlabel('epsilon')
# ax6.view_init(45,60)
ax6.legend()
fig6.show()

# test_acc
# x4 = np.asarray(test_mse)


# fig6 = plt_3d.figure(figsize=(10, 10))
# ax6 = fig6.add_subplot(111, projection='3d')
# img = ax6.scatter(x1, x2, x3, c=x4 ,cmap=plt_3d.hot())
# ax6.set_xlabel('gamma')
# ax6.set_ylabel('c')
# ax6.set_zlabel('epsilon')
# ax6.set_zlim([0,400])
# ax6.legend()
# cbar = fig6.colorbar(img)
# cbar.ax.set_ylabel('mean square error', rotation=270)

# ax6.legend()
# plt_3d.show()
            
# print(len(gammas))
# print(gammas)
print(epsilons)
# print(c_values)





In [ ]:
cross = []
i = 250
while i!=1500:
        cross.append(i)
        i = i+1
ax5.plot(cross,cross, color='black')
ax5.plot(y_test2,model_predict,'bo', markersize=3, color='blue')
# ax5.plot(y_train,model_predict_train,'bo', markersize=3,label='Train Data', color='red',marker = '<')
ax5.title('Cross plot for CS-VQ-SVR model')
ax5.xlabel('Actual value')
ax5.ylabel('Predicted value')
ax5.legend()
ax5.show()
# if __name__ == "__main__":
#     main()